## ODC-geo with H5NetCDF and earthaccess


In [1]:
import earthaccess
import xarray as xr
from common import earthaccess_args
from odc.geo.geobox import GeoBox
from odc.geo.xr import xr_reproject

In [ ]:
dataset = "gpm_imerg"
filename = earthaccess_args[dataset]["filename"]
bucket = earthaccess_args[dataset]["bucket"]
folder = earthaccess_args[dataset]["folder"]
variable = earthaccess_args[dataset]["variable"]

In [2]:
def warp_resample():
    input_uri = f"{folder}/{filename}"
    src = f"s3://{bucket}/{input_uri}"
    dstSRS = "EPSG:3857"
    srcSRS = "EPSG:4326"
    width = height = 256
    te = [
        -20037508.342789244,
        -20037508.342789244,
        20037508.342789244,
        20037508.342789244,
    ]
    earthaccess.login()
    fs = earthaccess.get_s3fs_session(daac="PODAAC")
    fsspec_caching = {
        "cache_type": "none",
    }
    gbox = GeoBox.from_bbox(te, dstSRS, shape=(height, width))
    f = fs.open(src, **fsspec_caching)
    da = xr.open_dataset(f, engine="h5netcdf", chunks={})[variable]
    da = da.odc.assign_crs(srcSRS)
    return xr_reproject(da, gbox)

In [ ]:
if __name__ == "__main__":
    da = warp_resample()